In [49]:
import pandas as pd
import numpy as np
import pickle

In [50]:
#data1=pd.read_csv("Car1.csv")
#data2=pd.read_csv("Car2.csv")
data=pd.read_csv("Car3.csv")

In [51]:
data.dtypes

Name            object
Price           object
Make            object
Model           object
Year            object
Condition       object
Transmission    object
Exchange        object
Fuel Type       object
BodyType        object
Location        object
Mileage         object
dtype: object

In [52]:
data.drop(['Exchange', 'BodyType'], axis=1, inplace=True)

In [53]:
data.drop("Name",axis=1,inplace=True)

In [54]:
data.dtypes

Price           object
Make            object
Model           object
Year            object
Condition       object
Transmission    object
Fuel Type       object
Location        object
Mileage         object
dtype: object

In [55]:
data=data[data["Year"]!="Year"]

In [56]:
data["Year"]=data["Year"].astype("i")

In [57]:
def clean_str(string):
    
    string=string.replace("was  6000000","").replace("km","").replace(",","").replace("₦","").replace("\n","").replace(",","")
    return string

In [58]:
data["Price"]=[clean_str(price) for price in data["Price"]]

In [59]:
data["Price"]=[clean_str(price) for price in data["Price"]]

In [60]:
data["Price"]=data["Price"].astype("f")

In [61]:
data["Location"]=[loc_string[-1] for loc_string in data["Location"].str.split(",")]

In [62]:
data["Location"]=[location.replace(" ","") for location in data["Location"]]

In [63]:
data["Mileage"].dtype

dtype('O')

In [64]:
def clean_mileage(mileage):
    try:
        return mileage.replace("km","").replace(",","")
    except:
            return mileage

In [65]:
data["Mileage"]=[clean_mileage(mileage) for mileage in data["Mileage"]]

In [66]:
data["Mileage"]=data["Mileage"].astype("f")

In [67]:
data["Transmission"]=data["Transmission"].replace("AUTOMATIC","Automatic")

In [68]:
data["Fuel Type"]=data["Fuel Type"].replace({"automatic":None,"manual":None,"Fuel":None})

In [69]:
data["Condition"]=data["Condition"].replace("red",None).replace("BLACK",None).replace("GREY",None).replace("WHITE",None).replace("Tokunbo","Foreign Used").replace({"Nigerian Used":"Registered"}).replace("Imported","Foreign Used")

In [70]:
data["Make"]=data["Make"].replace({"RAV4":"RAV-4","LandRover":"Land rover"})

In [71]:
data["Condition"].replace({"Registered":"Nigerian Used"},inplace=True)

In [72]:
data.dtypes

Price           float32
Make             object
Model            object
Year              int32
Condition        object
Transmission     object
Fuel Type        object
Location         object
Mileage         float32
dtype: object

In [73]:
data["Model"]=data["Model"].replace({"Cr-V":"CR-V","Lincoln":"licoln","Mdx":"MDX","RAV4":"Rav-4","RAV4":"RAV-4"})

In [74]:
data["Model"]=data["Model"].str.capitalize()

In [75]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.compose import ColumnTransformer

In [76]:
data.isna().sum()

Price           0
Make            0
Model           0
Year            0
Condition       0
Transmission    0
Fuel Type       0
Location        0
Mileage         0
dtype: int64

In [77]:
data.isna().sum()

Price           0
Make            0
Model           0
Year            0
Condition       0
Transmission    0
Fuel Type       0
Location        0
Mileage         0
dtype: int64

In [78]:
data.dtypes

Price           float32
Make             object
Model            object
Year              int32
Condition        object
Transmission     object
Fuel Type        object
Location         object
Mileage         float32
dtype: object

In [79]:
object_cols=[]
for i in data.columns:
    if data[i].dtype=="object":
        data[i]=data[i].astype("str")
        object_cols.append(i)

In [80]:
for i in data.columns:
    if data[i].dtype=="object":
        enc=LabelEncoder()
        data[i]=data[i].astype("str")
        data[i]=enc.fit_transform(data[i])

In [81]:
data

,Price,Make,Model,Year,Condition,Transmission,Fuel Type,Location,Mileage
0,2000000.0,25,12,2002,1,0,1,9,14000.0
1,3800000.0,25,58,2006,1,1,1,8,240000.0
2,2800000.0,25,20,2008,1,0,1,11,25000.0
3,2200000.0,25,28,2004,1,0,1,10,12000.0
4,1100000.0,25,70,2002,1,0,1,3,1000.0
...,...,...,...,...,...,...,...,...,...
1521,1350000.0,25,97,2002,0,0,1,5,185421.0
1522,35000000.0,3,6,2018,0,0,1,5,1755.0
1523,2200000.0,25,28,2005,0,0,1,3,1755.0
1524,6000000.0,8,29,2002,0,0,1,9,2000.0


In [82]:
target=data["Price"]
train=data.drop("Price",axis=1)

In [83]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler 
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import GradientBoostingRegressor

In [84]:
X,val,y,y_val=train_test_split(train,target,test_size=0.1,random_state=0)

In [85]:
target

0        2000000.0
1        3800000.0
2        2800000.0
3        2200000.0
4        1100000.0
           ...    
1521     1350000.0
1522    35000000.0
1523     2200000.0
1524     6000000.0
1525     4900000.0
Name: Price, Length: 1523, dtype: float32

In [86]:
data.shape

(1523, 9)

In [87]:
pipe=Pipeline([("imp",SimpleImputer(strategy="constant")),("scaler",StandardScaler()),("model",GradientBoostingRegressor(max_depth=5,random_state=42,max_features=5,learning_rate=0.025,verbose=2,n_estimators=1200))])

In [88]:
pipe.fit(X,y)

      Iter       Train Loss   Remaining Time 
         1 155977626087591296.0000            2.39s
         2 148276307440965728.0000            2.39s
         3 140955236152120496.0000            2.79s
         4 133995650639289360.0000            2.69s
         5 127379692398889008.0000            2.63s
         6 121090400858069568.0000            2.59s
         7 115111639227582448.0000            2.73s
         8 109428076480104288.0000            2.69s
         9 104025140488298752.0000            2.39s
        10 98888974506653856.0000            2.15s
        11 94006407767953056.0000            1.95s
        12 89364918515730896.0000            1.79s
        13 84952600867455600.0000            1.65s
        14 80758139545595920.0000            1.53s
        15 76770781546622240.0000            2.67s
        16 72980291850682416.0000            2.50s
        17 69376962438631304.0000            2.35s
        18 65951547634209568.0000            2.22s
        19 6269525261563036

       209 4886607654490.1172            1.76s
       210 4679785170969.0693            1.75s
       211 4484293747694.2256            1.74s
       212 4294113324906.7529            1.73s
       213 4116051683884.4565            1.72s
       214 3946778982217.5474            1.71s
       215 3785762011564.7759            1.70s
       216 3632579397638.7397            1.76s
       217 3487919103054.6597            1.75s
       218 3341731762225.5684            1.74s
       219 3209955607719.4795            1.73s
       220 3084615505101.4419            1.72s
       221 2964645414148.6909            1.71s
       222 2851333467920.9775            1.70s
       223 2743687365341.4365            1.69s
       224 2641822505972.0190            1.75s
       225 2544745234386.1641            1.74s
       226 2452509907508.2744            1.73s
       227 2363852278478.6196            1.72s
       228 2280537646728.1519            1.72s
       229 2200554048737.9756            1.71s
       230 21

       423 468023902004.2612            1.38s
       424 467660621668.5502            1.38s
       425 467339276730.9341            1.37s
       426 466808206999.3502            1.37s
       427 466160693608.0437            1.36s
       428 464587918254.9825            1.36s
       429 464294826793.9016            1.35s
       430 463759638792.1696            1.37s
       431 463059936177.6727            1.37s
       432 462135256554.6926            1.36s
       433 461100316424.4805            1.36s
       434 460837490265.4971            1.35s
       435 458839883196.6339            1.35s
       436 458328619284.3071            1.34s
       437 457189530675.4343            1.36s
       438 456484882901.0642            1.36s
       439 456314940045.2534            1.35s
       440 455506264723.4586            1.35s
       441 455225086847.0408            1.34s
       442 454880611908.3757            1.34s
       443 452476313702.4536            1.33s
       444 451712069742.4204      

       643 349064734593.7560            0.98s
       644 348789098712.9866            0.97s
       645 348448942538.6349            0.97s
       646 347310262369.6592            0.97s
       647 346352211193.7648            0.96s
       648 345274051239.6824            0.96s
       649 344910202708.3146            0.97s
       650 344398460264.3181            0.97s
       651 343576454427.3888            0.96s
       652 343472316995.7507            0.96s
       653 343392749943.0656            0.96s
       654 343016244941.3105            0.96s
       655 342819973528.0178            0.96s
       656 342742858314.0192            0.96s
       657 342028475780.7231            0.96s
       658 341956454106.5965            0.95s
       659 341886669501.9801            0.95s
       660 341567727891.9152            0.95s
       661 341093568676.1074            0.96s
       662 340862994167.0074            0.95s
       663 340613375491.6431            0.95s
       664 340221499027.2371      

       851 296825760363.3414            0.62s
       852 296739096375.6285            0.61s
       853 296554746081.4834            0.61s
       854 296418234646.9781            0.61s
       855 296339805543.5140            0.61s
       856 296133477854.1978            0.60s
       857 296058276178.4830            0.60s
       858 296047803137.2166            0.61s
       859 295908168926.3414            0.60s
       860 295639566643.1346            0.60s
       861 295519261972.1953            0.60s
       862 295464461856.4583            0.60s
       863 295342469520.8472            0.59s
       864 295090967869.8833            0.59s
       865 294893078130.9978            0.59s
       866 294717543515.6147            0.59s
       867 294431498879.6116            0.59s
       868 294375456145.7428            0.59s
       869 294251900725.5097            0.58s
       870 294186603494.8618            0.58s
       871 294145466386.2634            0.58s
       872 293741126395.3696      

      1073 272007591707.8994            0.22s
      1074 271914493838.7561            0.22s
      1075 271906197663.7319            0.22s
      1076 271662524322.9653            0.22s
      1077 271593343755.9634            0.21s
      1078 271533054512.0333            0.21s
      1079 271417934073.2164            0.21s
      1080 271392656410.2879            0.21s
      1081 271328607692.8518            0.21s
      1082 271198456411.1668            0.21s
      1083 271139912939.3250            0.20s
      1084 271011283423.8990            0.20s
      1085 270956373494.6732            0.20s
      1086 270885866014.4411            0.20s
      1087 270734881109.2343            0.20s
      1088 270645962919.6054            0.20s
      1089 270421908229.3001            0.19s
      1090 270417495316.8026            0.19s
      1091 270305159187.4764            0.19s
      1092 270278176340.1981            0.19s
      1093 270187753648.2408            0.19s
      1094 270183259391.7157      

Pipeline(steps=[('imp', SimpleImputer(strategy='constant')),
                ('scaler', StandardScaler()),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.025, max_depth=5,
                                           max_features=5, n_estimators=1200,
                                           random_state=42, verbose=2))])

In [89]:
pred=pipe.predict(val)

In [90]:
pipe.score(X,y)

0.999998395018688

In [91]:
pipe.score(val,y_val)

0.21189165766035634

In [92]:
pred_round=[round(val,-3) for val in pred]

In [93]:
mean_absolute_error(pred_round,y_val)

466261.4379084967

In [94]:

pickle.dump(pipe, open('model.pkl','wb'))

In [95]:
score=pipe.named_steps["model"].feature_importances_

In [96]:
scores=pd.DataFrame({"cols":train.columns,"scor":score}).sort_values(by="scor",ascending=False)

In [97]:
scores

,cols,scor
2,Year,6.377544e-01
1,Model,1.371453e-01
7,Mileage,8.839437e-02
6,Location,7.096664e-02
5,Fuel Type,6.279153e-02
0,Make,2.947156e-03
3,Condition,2.967423e-07
4,Transmission,2.887523e-07
